### Script pour le controle des données brutes issues de la chaine INSIGHT
### Etude de conformité des fichiers

In [2]:
%%writefile ".env"

PROJECT_ID="feux_cq"

COMMUN_PATH = "N:/"
PATH_INFOCENTRE_APP = ${COMMUN_PATH}Informatique/SIG/Application/Jupyterhub/
PATH_ETUDE = ${COMMUN_PATH}Informatique/SIG/Donnees/Oeil/Traitement_Donnees/SURFACES_BRULEES_SENTINEL/2022/220502_PreparationFeu2020/
PROJECT_PATH =${COMMUN_PATH}Informatique/SIG/Etudes/2023/2309_QC_feux/Travail/Scripts/
DATA_CATALOG_DIR = ${PATH_INFOCENTRE_APP}projets/catalogFiles/ 
DATA_OUTPUT_DIR = ${PROJECT_PATH}output/
SIG_DATA_PATH = ${COMMUN_PATH}Informatique/SIG/Donnees/
DB_USER="jfnguyenvansoc"
DB_PWD="oeil"
DB_HOST=172.20.12.13
DB_PORT=5432

DB_WORKSPACE="oeil_traitement"
DB_REF="oeil_reference"
DB_EXT="data_externe"
DB_SCHEMA = "feux_cq"
DB_SCHEMA_REF = "carto"

Overwriting .env


In [1]:
import logging
from filecmp import dircmp
from os import listdir
import os
import filecmp
from dotenv import load_dotenv
from intake import open_catalog
import matplotlib as plt
import pandas as pd
import numpy as np
from shapely import wkt
import shapely
from sqlalchemy import create_engine
from shapely.geometry import shape
from shapely.ops import unary_union
from geopandas import GeoDataFrame
import geopandas as gpd
from datetime import datetime

load_dotenv()

## Create log file 
logging.basicConfig(filename='N:/Informatique/SIG/Etudes/2023/2309_QC_feux/Travail/Scripts/Controle_data/log_Controle_data_brute.log',filemode='w', encoding='utf-8', level=logging.INFO, force=True, format='%(message)s')

##open yaml file to get raw buned area table
catfeux = open_catalog(f'{os.getenv("PROJECT_PATH")}catalogFiles/Fire_Detection_Data_Quality.yaml')

surfdetect_control = catfeux.Surfaces_brulees_brute_control.read()

date_start=pd.to_datetime("2023-01-01", format="%Y-%m-%d")
date_end=pd.to_datetime("2023-12-31", format="%Y-%m-%d")

#date_start=datetime.strptime("2023-01-01","%Y-%m-%d") 
#date_end=datetime.strptime("2023-10-31","%Y-%m-%d")

C:\Users\oriane.bruyere\AppData\Local\Temp\ipykernel_12472\3508062948.py:16: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  from geopandas import GeoDataFrame


## Début des contrôles

In [3]:
## create a table to integrate generalities about control steps

table_control_number=[]

### Check if tile's name are differents than the 15 use in NC

In [ ]:

logging.info('------------------------------------------------')
logging.info('Check if unknown tiles are found in data')
logging.info('------------------------------------------------')

## liste des 15 tuiles présentes sur la GT et les iles 
list_tiles=['L2A_T58KCC','L2A_T58KCD','L2A_T58KDB','L2A_T58KDC','L2A_T58KEA','L2A_T58KEB','L2A_T58KEC',
            'L2A_T58KFA','L2A_T58KFB','L2A_T58KFC','L2A_T58KGA','L2A_T58KGB','L2A_T58KGC','L2A_T58KGV','L2A_T58KHB']

## prendre que le numéro de la dalle pour les traitements
surfdetect_control['nom'] = [x[-10:] for x in surfdetect_control['nom']]

error_tile=[]

for i in range(len(surfdetect_control)):
    if surfdetect_control['nom'][i] in list_tiles:
         continue
    else:
        error_tile.append(surfdetect_control.iloc[i]) ## on regarde si d'autres tuiles sont présentes dans le jeu de donnée

if not error_tile:
    print('Aucune tuiles détectées en dehors des 15 tuiles utilisées par l algorithme')
    logging.info('Not unkonw tiles found in data - continue process')
    len_error_tile=0
else:
    error_tile=pd.concat(error_tile,axis=1, ignore_index=True).T
    print('Attention des tuiles en dehors des tuiles abituelles sont présentes dans le jeux de donnée')
    logging.error('%s !! WARNING !! Unknow tile found in data - check ogc_fid number above or in code !! :', error_tile['ogc_fid']) ## on alimente le log si présence de tuiles autres
    len_error_tile=len(error_tile)

table_control_number.append(len_error_tile)

### Check if tile occurence is equal to zero

In [ ]:
### Check if tile occurence is equal to zero
## traitement à prendre avec des pincettes car selon la résolution temporelle des données des tuiles peuvent manquer 

logging.info('------------------------------------------------')
logging.info('Check if tile occurence is set to 0')
logging.info('------------------------------------------------')

df_dalle=pd.DataFrame({"date":surfdetect_control.date,"dalle_names":surfdetect_control.nom})
## prendre que le numéro de la dalle
df_dalle['dalle_names'] = [x[-10:] for x in df_dalle['dalle_names']]
df_dalle=df_dalle.set_index(df_dalle['date'])
df_dalle = df_dalle.sort_index()

occurrences_dalles = df_dalle["dalle_names"].value_counts()
print(occurrences_dalles)

values_at_zero = occurrences_dalles[occurrences_dalles == 0].index

if not values_at_zero.empty:
    logging.error('%s !! WARNING !! Tile occurence is set to 0 - check tile name above or in code !! :', values_at_zero)
    print(values_at_zero)
else:
    logging.info('Tile occurence is different than 0 in data - continue process')

occurrences_dalles.to_csv("tile_occurence.csv")

### Check if burned area are under 1ha

In [ ]:
## Check if burned area are under 1ha

logging.info('------------------------------------------------')
logging.info('Check if burned areas are not under 1 ha')
logging.info('------------------------------------------------')

error_surface=[]

for i in range(len(surfdetect_control)):
    if surfdetect_control['surface'][i] >= 1:
         continue
    else:
        error_surface.append(surfdetect_control.iloc[i])

if not error_surface:
    print('empty_list')
    logging.info('No surface found under 1ha - continue process')
    len_error_surface=0
else:
    error_surface=pd.concat(error_surface,axis=1, ignore_index=True).T
    print('list if not empty')
    logging.error('%s !! WARNING !! burned areas are found under 1 ha - check ogc_fid number above or in code !! :', error_surface['ogc_fid'])
    len_error_surface=len(error_surface)
    
table_control_number.append(len_error_surface)

### Check if geometries have errors

In [4]:
## Check if burned area have wrong geometries

logging.info('------------------------------------------------')
logging.info('Check if burned areas have wrong geometries')
logging.info('------------------------------------------------')

is_valid = surfdetect_control.geometry.is_valid
invalid_geometries = surfdetect_control[~is_valid]
len_invalid_geometries=len(invalid_geometries)

print("Multipolygones with geometries errors :",invalid_geometries)
logging.error('%s !! WARNING !! burned areas geometries have errors - check ogc_fid number above or in code !! :', invalid_geometries.index)

surfdetect_control.geometry = surfdetect_control.geometry.buffer(0) # correct the geometry

## check if the correction is applied
is_valid_after_correction = surfdetect_control.geometry.is_valid
invalid_geometries_after_correction = surfdetect_control[~is_valid_after_correction]
print("Check if geometry correction were applied :", invalid_geometries_after_correction)
logging.error('%s !! WARNING !! a geometry correction were applied - if empty continue process : ',invalid_geometries_after_correction)

table_control_number.append(len_invalid_geometries)

Multipolygones with geometries errors :            date                                        nom       province  \
143    20230103  SENTINEL2B_20230103-231232-056_L2A_T58KEA   Province Sud   
1810   20230222  SENTINEL2B_20230222-231219-065_L2A_T58KEB  Province Nord   
3667   20230503  SENTINEL2B_20230503-231221-618_L2A_T58KEB  Province Nord   
3809   20230503  SENTINEL2B_20230503-231221-618_L2A_T58KEB  Province Nord   
14188  20230730  SENTINEL2A_20230730-232215-863_L2A_T58KEB  Province Nord   
14957  20230730  SENTINEL2A_20230730-232215-863_L2A_T58KEB  Province Nord   
15466  20230801  SENTINEL2B_20230801-231224-016_L2A_T58KEB  Province Nord   
16385  20230806  SENTINEL2A_20230806-231209-332_L2A_T58KEC  Province Nord   
17458  20230811  SENTINEL2B_20230811-231224-657_L2A_T58KEB  Province Nord   
18738  20230814  SENTINEL2B_20230814-232157-720_L2A_T58KCD  Province Nord   
18815  20230814  SENTINEL2B_20230814-232223-144_L2A_T58KDB  Province Nord   
19366  20230816  SENTINEL2A_20230816

#### Check if burned areas are detected outside date interval set at the begening

In [ ]:
## Check if burned areas are detected outside date interval set at the begening

logging.info('------------------------------------------------')
logging.info('%s Check if burned areas are included into the following date interval', )
logging.info('------------------------------------------------')

error_date=[]

for i in range(len(surfdetect_control)):
    current_date = pd.to_datetime(surfdetect_control['date'][i])
    
    if date_start <= current_date <= date_end:
        continue
    else:
        error_date.append(surfdetect_control.iloc[i])

if not error_date:
    print('empty_list')
    logging.info('Burned areas are included into date range - continue process')
    len_outside_date=0
else:
    error_date=pd.concat(error_date,axis=1, ignore_index=True).T
    print('list if not empty')
    logging.error('%s !! WARNING !! burned areas are found outside date range - check ogc_fid number above or in code !! :', error_date['ogc_fid'])
    len_outside_date=len(error_date)

table_control_number.append(len_outside_date)

In [5]:
## create an id
surfdetect_control['ID'] = range(1, len(surfdetect_control)+1,1)

In [8]:
surfdetect_control.head(10)

,date,nom,province,commune,surface,x,y,cuirasse,cuira_area,bdtopo,bdtop_area,layer,path,geometry,ID
0,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,POUEBO,1.221119,249183,423465,None,NaN,None,NaN,None,None,"POLYGON ((249215.650 423586.246, 249235.659 42...",1
1,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,OUEGOA,1.561495,233428,436165,None,NaN,None,NaN,None,None,"POLYGON ((233394.382 436298.560, 233404.386 43...",2
2,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,POUEBO,1.981800,249826,421880,None,NaN,None,NaN,None,None,"POLYGON ((249786.292 421999.216, 249796.297 42...",3
3,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,POUEBO,1.100996,249941,421203,None,NaN,None,NaN,None,None,"POLYGON ((249960.681 421340.042, 249970.685 42...",4
4,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,OUEGOA,1.931640,230088,420823,None,NaN,None,NaN,None,None,"POLYGON ((230184.050 420960.118, 230194.054 42...",5
5,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,KAALA GOMEN,1.110744,246885,374413,None,NaN,None,NaN,None,None,"POLYGON ((246874.227 374510.718, 246904.237 37...",6
6,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,KAALA GOMEN,1.651067,245792,368610,None,NaN,None,NaN,None,None,"POLYGON ((245751.303 368731.413, 245761.306 36...",7
7,20230101,SENTINEL2A_20230101-232202-326_L2A_T58KDC,Province Nord,VOH,1.491013,261963,368525,None,NaN,None,NaN,None,None,"POLYGON ((261877.551 368645.668, 261887.554 36...",8
8,20230101,SENTINEL2A_20230101-232216-649_L2A_T58KDB,Province Nord,KAALA GOMEN,1.380929,247296,374837,None,NaN,None,NaN,None,None,"POLYGON ((247201.800 374902.982, 247211.803 37...",9
9,20230101,SENTINEL2A_20230101-232216-649_L2A_T58KDB,Province Nord,KAALA GOMEN,1.030690,246888,374415,None,NaN,None,NaN,None,None,"POLYGON ((246874.227 374510.718, 246904.237 37...",10


### Check if identical geometries are observed at a same date

In [ ]:
#check if identical geometries are observed at a same date
len_surfdetect_control_before=len(surfdetect_control)
def compare_geometries(gdf):
    geometry_date_dict = {}

    for index, row in gdf.iterrows():
        geometry =  row['geometry']
        date = row['date']

        # Convertir la géométrie en une forme Shapely
        geom_shape = shape(geometry)
        key = (geom_shape, date)

        # Ajouter la paire (forme, date) au dictionnaire
        if key not in geometry_date_dict:
            geometry_date_dict[key] = []

        geometry_date_dict[key].append(index)
    identique_geom=[]
    
    # identifier les géométries avec la même forme à la même date = doublon
    for key, indices in geometry_date_dict.items():
        if len(indices) > 1:
            identique_geom.append(indices)
            print(f"Géométries identiques à la même date : {indices}")
    
    return(identique_geom)

surfdetect_control['geometry']=shapely.wkt.loads(shapely.wkt.dumps(surfdetect_control['geometry'], rounding_precision=1)) ## change number of decimal for geometry
identique_geom=compare_geometries(surfdetect_control) 
print(identique_geom)

logging.info('------------------------------------------------')
logging.info('%s Check if 2 identical geometries shape are observed at a same date', )
logging.info('------------------------------------------------')

if not identique_geom:
    print('empty_list')
    logging.info('NO identical geometries found at the same date - continue process')
    len_identique_geom=0
else:
    print('list if not empty')
    len_identique_geom=len(identique_geom)
    #logging.error('%s !! WARNING !! same geometries at a same date were found - check ogc_fid number above or in code !! :', identique_geom.index)
    
    ### if identical geometries are found : check if they have the name tile name
    for sublist in identique_geom:
        nom_values = [surfdetect_control.loc[i, 'nom'] for i in sublist]
        index_values=[surfdetect_control.loc[i, 'ID'] for i in sublist]
        if all(value == nom_values[0] for value in nom_values):
            print('nom de thuile identique')
            logging.error('%s !! WARNING !! same geometries at a same date were found on a same tile: check above for informations',index_values)
            
            ## we keep the first geom et delete the others
            if len(sublist) == 2: 
                surfdetect_control=surfdetect_control.drop(sublist[1])
                print('delete index : ',sublist[1])
            else:
                surfdetect_control=surfdetect_control.drop(sublist[1:])
                print('delete index :',sublist[1:])
        else:
            print('nom de thuile différente')
            logging.error('%s !! WARNING !! same geometries at a same date were found on different tiles : check above for informations',index_values)
            
            ## we keep the first geom et delete the others
            if len(sublist) == 2: 
                surfdetect_control=surfdetect_control.drop(sublist[1])
                print('delete index : ',sublist[1])
            else:
                surfdetect_control=surfdetect_control.drop(sublist[1:])
                print('delete index :',sublist[1:])

#surfdetect_control=surfdetect_control.reset_index(drop=True)
#surfdetect_control['ID'] = range(1, len(surfdetect_control)+1,1)

table_control_number.append(len_identique_geom)
table_control_number.append(len_surfdetect_control_before-len(surfdetect_control))

In [ ]:
surfdetect_control

## Overlapping polygons for a same date

In [ ]:
def find_intersecting_id(row, gdf):
    same_date_gdf = gdf[gdf['date'] == row['date']]
    
    # Trouver les identifiants des polygones s'intersectant avec le polygone actuel
    intersecting_ids = same_date_gdf[same_date_gdf.geometry.overlaps(row['geometry'])]['ID'].tolist()
    intersecting_ids = [id_ for id_ in intersecting_ids if id_ != row['ID']]
    
    return intersecting_ids

surfdetect_control['Intersecting_ID'] = surfdetect_control.apply(lambda row: find_intersecting_id(row, surfdetect_control), axis=1)
surfdetect_control['groupe_id'] = None
groupe_id_counter = 0

# attribuer des groupe_id à tous les polygones 
for index, row in surfdetect_control.iterrows():
    if surfdetect_control.at[index, 'groupe_id'] is None:
        # Attribuer le groupe_id actuel à tous les polygones s'intersectant
        current_group_id = groupe_id_counter
        surfdetect_control.loc[(surfdetect_control['Intersecting_ID'].apply(lambda x: row['ID'] in x)) | (surfdetect_control['ID'] == row['ID']), 'groupe_id'] = current_group_id
        
        groupe_id_counter += 1

### Check if overlapping geometries are around tile's limit

In [ ]:
tile_stl2 = catfeux.tile_sentinel2_line_UTM.read() ## ouvirr la table des limites des tuiles Sentinel2
tile_stl2=tile_stl2.set_index(tile_stl2['Name']) ## mettre en index le nom de la tuile

group_ids = surfdetect_control['groupe_id'].unique()

logging.info('------------------------------------------------')
logging.info('%s Check if overlapping geometries at a same date are observed close to tile limits ', )
logging.info('------------------------------------------------')

for group_id in group_ids:
    # Sélectionnez les données pour le groupe en cours
    group_data = surfdetect_control[surfdetect_control['groupe_id'] == group_id]
    group_data['nom']= [x[-5:] for x in group_data['nom']]
    if group_data.shape[0] >= 2 :

        for i in range(len(group_data)):
            if group_data.iloc[i]['geometry'].buffer(20).intersects(tile_stl2['geometry'].loc[group_data.iloc[i]['nom']]):
                print("tile touche",group_data.iloc[i]["ID"]-1)
                surfdetect_control.drop(group_data.iloc[i]["ID"]-1,inplace=True)
                logging.error('%s !! WARNING !! Intersection between geometry and tile limits were found : check above for informations',group_data.iloc[i]["ID"]-1)

### Check if overlapping geometries are from differents tiles

In [ ]:
group_ids = surfdetect_control['groupe_id'].unique()

logging.info('------------------------------------------------')
logging.info('%s Check if overlapping geometries at a same date are observed from a same tile ', )
logging.info('------------------------------------------------')

for group_id in group_ids:
    
    group_data = surfdetect_control[surfdetect_control['groupe_id'] == group_id]
    group_data['nom']= [x[-5:] for x in group_data['nom']]
    if group_data.shape[0] >= 2 :
        if all(v == group_data['nom'].iloc[0] for v in group_data['nom']):
            print(f"les noms des tuiles sont identiques.", group_data)
            logging.error('%s !! WARNING !! overlapping geometries at a same date were found on a same tile : check above for informations',group_data)
        
        else: ## we merge geometries overlapping from differents tiles
            print(f"les noms des tuiles sont différents")
            nouveau_polygone = unary_union(group_data['geometry'])
            surfdetect_control.at[group_data.iloc[0]["ID"]-1,'geometry'] = nouveau_polygone
            surfdetect_control.drop(group_data.iloc[1:]["ID"]-1,inplace=True)

In [ ]:
surfdetect_control=surfdetect_control.reset_index(drop=True)
surfdetect_control['ID'] = range(1, len(surfdetect_control)+1,1)

In [ ]:
from shapely.geometry import MultiPolygon

# Vérifier si la géométrie est de type Polygon
for i in range(len(surfdetect_control)):
    if surfdetect_control['geometry'].geom_type[i] == 'Polygon':
        surfdetect_control.at[i, 'geometry'] = MultiPolygon([surfdetect_control['geometry'][i]])
        #print(surfdetect_control)
    else:
        print("La géométrie n'est pas de type Polygon.")

In [ ]:
surfdetect_control['surface'] = surfdetect_control['geometry'].area/10000

### Control with cloud cover dynamic : use of STAC Catalog

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.dates import MonthLocator, DateFormatter

In [ ]:
## time serie of burned area detected by day
surfdetect_control['date'] = pd.to_datetime(surfdetect_control['date'])
surfdetect_control=surfdetect_control.set_index(surfdetect_control['date'])

daily_counts = surfdetect_control.resample('D').size()

plt.figure(figsize=(12, 6))
plt.bar(daily_counts.index, daily_counts, color='b')
plt.xlim(date_start,date_end)
plt.title("Série temporelle du nombre de détections par jours en 2023")
plt.grid(True)

### Plot ocurence for each tile

In [ ]:
grouped = surfdetect_control.groupby('nom')
date_start=pd.to_datetime("2023-01-01", format="%Y-%m-%d")
date_end=pd.to_datetime("2023-12-31", format="%Y-%m-%d")

fig, axs = plt.subplots(nrows=5,ncols=3, figsize=(15, 13))
for i, (name, group_df) in enumerate(grouped):
    group_df['date'] = pd.to_datetime(group_df['date'])
    date_counts = group_df.resample('D').size()
    
    row_idx, col_idx = divmod(i, 3)
    
    axs[row_idx, col_idx].bar(date_counts.index, date_counts.values, width=0.8, align='center', color='blue', edgecolor='black')
    axs[row_idx, col_idx].set_title(f'Séries temporelles pour {name}')
    axs[row_idx, col_idx].set_xlabel('Date')
    axs[row_idx, col_idx].set_ylabel('Nombre d\'occurrences')
    axs[row_idx, col_idx].set_xlim(date_start, date_end)
    axs[row_idx, col_idx].xaxis.set_major_locator(MonthLocator())
    axs[row_idx, col_idx].xaxis.set_major_formatter(DateFormatter('%b'))
    axs[row_idx, col_idx].grid(True, which='both', linestyle=':', linewidth='0.5')

plt.tight_layout()
plt.show()

### Extract cloud cover from STAC Catalog

In [ ]:
from pystac_client import Client
catalog = Client.open("https://earth-search.aws.element84.com/v1")
query = catalog.search(
    collections=["sentinel-2-l2a"],datetime="2023-05-01/2023-05-31", bbox=[163.362, -22.76, 168.223, -19.479]
)

In [ ]:
from sentinelhub import SHConfig

config = SHConfig()
config.instance_id = "stac"
config.sh_client_id = '713157aa-ecbb-44fe-a8e6-e6e0af3eb968'
config.sh_client_secret = 'VGwoQEb8ye3aQWl1K03WFDjai3qae0kY'
config.save("my-profile")

catalog = SentinelHubCatalog(config=config)

catalog.get_info()
collections = catalog.get_collections()

collections = [collection for collection in collections if not collection["id"].startswith(("byoc", "batch"))]

In [ ]:
caspian_sea_bbox = BBox((163.362, -22.76, 168.223, -19.479), crs=CRS.WGS84)
time_interval = "2023-01-01", "2023-12-31"

search_iterator = catalog.search(
    DataCollection.SENTINEL2_L2A,
    bbox=caspian_sea_bbox,
    time=time_interval,
    fields={"include": ["id", "properties.datetime", "properties.eo:cloud_cover"], "exclude": []},
)

results = list(search_iterator)
print("Total number of results:", len(results))

results

In [ ]:
df = pd.json_normalize(results)
df = df.rename(columns={
    'id': 'ID',
    'properties.datetime': 'Date',
    'properties.eo:cloud_cover': 'Cloud_Cover'
})

df['ID'] = [x[39:-16] for x in df['ID']]
included_names = ['58KDB', '58KDC','58KEB','58KFA']
df = df[df['ID'].isin(included_names)]

replacements = {'58KDB': 'L2A_T58KDB', '58KDC': 'L2A_T58KDC', '58KEB': 'L2A_T58KEB','58KFA':'L2A_T58KFA'}
df.replace(replacements, inplace=True)
df=df.reset_index(drop=True)
df['Date'] = pd.to_datetime(df['Date'], utc=True)

# Extraire la partie date (année, mois, jour) en format YYY-MM-DD
df['Date'] = df['Date'].dt.strftime('%Y-%m-%d')

#### Zoom on famous tiles

In [ ]:
included_names = ['L2A_T58KDB', 'L2A_T58KDC','L2A_T58KEB','L2A_T58KFA']
from matplotlib.colors import Normalize
import matplotlib.colors as mcolors
custom_cmap = mcolors.ListedColormap(['green', 'blue', 'red'])

fig, axs = plt.subplots(nrows=2,ncols=2, figsize=(15, 8))
for i, name in enumerate(included_names):
    cc_group = df.groupby('ID')

    if name in grouped.groups:
        group_df = grouped.get_group(name)
        cc_group = cc_group.get_group(name)

        group_df['date'] = pd.to_datetime(group_df['date'])
        date_counts = group_df.resample('D').size()
        row_idx, col_idx = divmod(i, 2)
        array_50 = np.full(len(cc_group.Date), -40)

        colors = np.zeros_like(cc_group.Cloud_Cover, dtype='str')
        colors[(cc_group.Cloud_Cover >= 0) & (cc_group.Cloud_Cover< 30)] = 'green'
        colors[(cc_group.Cloud_Cover >= 30) & (cc_group.Cloud_Cover < 60)] = 'blue'
        colors[cc_group.Cloud_Cover >= 60] = 'red'

        axs[row_idx, col_idx].axhline(y=0,lw=0.5,c='k')
        axs[row_idx, col_idx].bar(date_counts.index, date_counts.values, width=1.5, align='center', color='k')
        scatter=axs[row_idx, col_idx].scatter(cc_group.Date,array_50,c=colors,vmin=0,vmax=100, s=600,marker='|',linewidth=2)
        axs[row_idx, col_idx].set_title(f'Séries temporelles pour {name}')
        axs[row_idx, col_idx].set_xlabel('Date')
        axs[row_idx, col_idx].set_ylabel('Nombre d\'occurrences')
        axs[row_idx, col_idx].set_xlim(date_start, date_end)
        axs[row_idx, col_idx].xaxis.set_major_locator(MonthLocator())
        axs[row_idx, col_idx].xaxis.set_major_formatter(DateFormatter('%b'))
        axs[row_idx, col_idx].grid(True, which='both', linestyle=':', linewidth='0.5')
        axs[row_idx, col_idx].set_ylim(-50,550+10)

fake_handles = [plt.Line2D([0], [0], marker='s', markerfacecolor=color, markersize=10) for color in ['green', 'blue', 'red']]
legend = plt.legend(fake_handles, ['0-30', '30-60', '>60'], title='Cloud Cover (%)',fontsize=16, loc='upper right', frameon=False)

# Suppression du contour de la légende

plt.tight_layout()
plt.show()

## FINALISATION : Intégration des données dans la table "sentinel_surfaces_detectees"

In [ ]:
def IntegrationSurfacesSentinelGPK(s, inf, project_db_schema,data, name_regr, conn, filter_date, chmps_s, chmps_orig):
    data.to_postgis(name_regr,conn, schema=project_db_schema,if_exists='replace')
    sqlintegr = f"INSERT INTO {project_db_schema}.{s} ({chmps_s}) " + \
        f"SELECT {chmps_orig} FROM {project_db_schema}.{name_regr}"
    conn.connect().execute(sqlintegr)
    #insertion via table informations sentinel des nouveux identifiants
    sql_upd_info = f"INSERT INTO {project_db_schema}.{inf} (ogc_fid_surface, date_) " + \
        f"SELECT ogc_fid, date_ from {project_db_schema}.{s} where date_ >= '{filter_date}'"
    conn.connect().execute(sql_upd_info)

In [ ]:
s = "sentinel_surfaces_detectees"
inf = "sentinel_informations_surfaces_detectees"
datetxtdb = "date_texte"
datedb = "date_"
dateorig = '"date"'
chps = "nom, province, commune, surface, x, y, cuirasse, cuira_area, bdtopo, bdtop_area, geometry"
champs_s = f'{datetxtdb}, {chps}, {datedb}'
format_date_to_timestamp = 'YYYYMMDD'
champs_s_orig = f"{dateorig}, {chps}, to_timestamp({dateorig}, '{format_date_to_timestamp}')"
filt_date = '2023-1-1'
name_couche_temp="sentinel_surfaces_detectees_2023"

In [ ]:
conex = create_engine(f'postgresql://{os.getenv("DB_USER")}:{os.getenv("DB_PWD")}@{os.getenv("DB_HOST")}:{os.getenv("DB_PORT")}/{os.getenv("DB_WORKSPACE")}')
IntegrationSurfacesSentinelGPK(s, inf, os.getenv("DB_SCHEMA"),surfdetect_control , name_couche_temp, conex, filt_date, champs_s, champs_s_orig)
conex.dispose()